In [191]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import pandas as pd
import numpy as np
from math import ceil, log

from bokeh.layouts import column, row, widgetbox, Spacer, layout
from bokeh.io import push_notebook, show, output_notebook
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models import ColumnDataSource, Select, LabelSet, Label
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [123]:
#implement class plotter that plots everything that is necessary
class plotter:
    def __init__(self, _aggr, _depl, _Rss, _S_0,
                 _kvalues=np.ones(7), _kmax=0, _wire_url=0, _formula_url=0,
                _N=200, _Rmax=2, _Rmin=0, _Smax=3, _Smin=0, _title='test', _imgheight=200):
        self.aggregation = _aggr
        self.depletion = _depl
        self.Rss = _Rss
        self.kvalues = _kvalues
        self.N = _N #density of plot points
        self.Rmax = _Rmax
        self.Rmin = _Rmin
        self.Smax = _Smax
        self.Smin = _Smin
        self.S = _S_0
        S0 = [_S_0] #this is necessary since some struct need a list as input
        self.wire_url = _wire_url
        self.formula_url = _formula_url
        self.title = _title
        if (np.size(_kmax)==1):
            self.kmax=np.array(_kvalues)
            self.kmax[:]['value'] = self.kmax[:]['value']*2
        else:
            self.kmax=_kmax
            
        self.imgheight=_imgheight
        
        # Set up data
        R_x = np.linspace(self.Rmin, self.Rmax, self.N)
        self.dRmin = np.max(self.depletion(R_x, self.S, self.kvalues))
        dR_dt_depl = self.depletion(R_x, self.S, self.kvalues)
        dR_dt_aggr = self.aggregation(R_x, self.S, self.kvalues)
        R_ss = self.Rss(self.S, self.kvalues)
        dR_dt_ss = self.depletion(R_ss, self.S, self.kvalues)
        self.dRmax = max(np.max(dR_dt_depl), np.max(dR_dt_aggr))
        self.dRmin = min(np.min(dR_dt_depl), np.min(dR_dt_aggr))
        
        self.data_dR_dt_R = ColumnDataSource(data=dict(R=R_x, dR_dt_depl=dR_dt_depl, dR_dt_aggr=dR_dt_aggr)) #graphs
        self.data_dR_dt_R_lines = ColumnDataSource(data=dict(R_ss_y=[R_ss[0], R_ss[0]], dR_dt_ss_y=[0, self.depletion(R_ss[0], self.S, self.kvalues)]))#y parallel
        self.data_dR_dt_R_point_lab = ColumnDataSource(data=dict(R_ss=[R_ss], dR_dt_ss=[dR_dt_ss], lab=["R_ss = %.2f" % R_ss])) #label
        self.data_dR_dt_R_point = ColumnDataSource(data=dict(R_ss=R_ss, dR_dt_ss=dR_dt_ss)) #label
        #we need to separate this, since bokeh has problem with drawing circles from data in the label structure
        
        S = np.linspace(self.Smin, self.Smax, self.N)
        R_y = self.Rss(S, self.kvalues)
        self.data_S_R = ColumnDataSource(data=dict(S=S, R=R_y))
        self.data_S_R_lines = ColumnDataSource(data=dict(R_ss_y=[0, R_ss[0]], S_ss_y=[self.S, self.S], #y and x parallel
                                                    R_ss_x=[R_ss[0], R_ss[0]], S_ss_x=[0, self.S]))
        self.data_S_R_point = ColumnDataSource(data=dict(S_ss=S0, R_ss=R_ss))#dot
        
        # Set up widgets
        self.S_var = Slider(title="S", value=self.S, start=self.Smin, end=self.Smax, step=(self.Smax - self.Smin)/100)
        self.k_var = list(np.empty(np.size(self.kvalues)))
        for i, k in enumerate(self.kvalues):
            self.k_var[i] = Slider(title=k['name'], value=k['value'], start=0.0, end=self.kmax[i]['value'], step=0.01)
    
    def create_figure(self):
        # Set up plot
        #plot_dR_dt = figure(plot_height=600, plot_width=600, title="Depletion/Aggregation Rate",
        #      #tools="crosshair,pan,reset,save,wheel_zoom",
        #      x_range=[self.Rmin, self.Rmax], y_range=[self.dRmin, self.dRmax], toolbar_location="above")

        plot_dR_dt = figure(plot_height=800, plot_width=600, title="Depletion/Aggregation Rate",
              x_range=[self.Rmin, self.Rmax], y_range=[self.dRmin, self.dRmax])
        
        
        plot_dR_dt.line('R', 'dR_dt_depl', source=self.data_dR_dt_R, line_width=3, line_alpha=0.6, color='crimson')
        plot_dR_dt.line('R', 'dR_dt_aggr', source=self.data_dR_dt_R, line_width=3, line_alpha=0.6, color='steelblue')
        plot_dR_dt.line('R_ss_y', 'dR_dt_ss_y', source=self.data_dR_dt_R_lines, line_width=3, line_alpha=0.6, color='red', line_dash='4 4')
        
        labels = LabelSet(x='R_ss', y='dR_dt_ss', text='lab', level='glyph',
              x_offset=7, y_offset=-25, source=self.data_dR_dt_R_point_lab, render_mode='canvas')
        plot_dR_dt.add_layout(labels)
        
        plot_dR_dt.circle('R_ss', 'dR_dt_ss', source=self.data_dR_dt_R_point, fill_color="white", size=10)
        
        plot_dR_dt.yaxis.axis_label = "dR/dt"
        plot_dR_dt.xaxis.axis_label = "R"
        
        plot_dR_dt.toolbar.logo = None
        plot_dR_dt.toolbar_location = None
        
            
            
        plot_R_S= figure(plot_height=800, plot_width=600, title="Steady State Solutions",
                  #tools="crosshair,pan,reset,save,wheel_zoom",
                  x_range=[self.Smin, self.Smax], y_range=[self.Rmin, self.Rmax], toolbar_location="above")
    
        plot_R_S.line('S', 'R', source=self.data_S_R, line_width=3, line_alpha=0.6, color='black')
        plot_R_S.line('S_ss_y', 'R_ss_y', source=self.data_S_R_lines, line_width=3, line_alpha=0.6, color='black', line_dash='4 4')
        plot_R_S.line('S_ss_x', 'R_ss_x', source=self.data_S_R_lines, line_width=3, line_alpha=0.6, color='red', line_dash='4 4')
        
        plot_R_S.circle(x='S_ss', y='R_ss', source=self.data_S_R_point, fill_color="white", size=10)
        
        plot_R_S.yaxis.axis_label = "R_ss"
        plot_R_S.xaxis.axis_label = "S"
        
        plot_R_S.toolbar.logo = None
        plot_R_S.toolbar_location = None
        
        plot_wire = figure(plot_height=300, plot_width=300, x_range=(0,1), y_range=(0,1), title='Wire Diagramm')
        plot_wire.image_url(url=[self.wire_url], x=0, y=1, w=1, h=1)
        
        plot_wire.toolbar.logo = None
        plot_wire.toolbar_location = None
        plot_wire.axis.visible = False
        
        plot_formula = figure(plot_width=300, plot_height=self.imgheight, x_range=(0,1), y_range=(0,1), title='Formulas')
        plot_formula.image_url(url=[self.formula_url], x=0.01, y=0.99, w=0.98, h=0.98)
        
        plot_formula.toolbar.logo = None
        plot_formula.toolbar_location = None
        plot_formula.axis.visible = False
        
        return plot_dR_dt, plot_R_S, plot_wire, plot_formula 
        
    def update_data(self, attrname, old, new):
    
        # Get the current slider values
        self.S = self.S_var.value
        S0 = [self.S]
        for i,k in enumerate(self.k_var):
             self.kvalues['value'][i] = k.value
    
        ## update curve
        R_x = np.linspace(self.Rmin, self.Rmax, self.N)
        dR_dt_depl = self.depletion(R_x, self.S, self.kvalues)
        dR_dt_aggr = self.aggregation(R_x, self.S, self.kvalues)
        R_ss = self.Rss(self.S, self.kvalues)
        
        self.data_dR_dt_R.data = dict(R=R_x, dR_dt_depl=dR_dt_depl, dR_dt_aggr=dR_dt_aggr) #graphs
        self.data_dR_dt_R_lines.data = dict(R_ss_y=[R_ss[0], R_ss[0]], dR_dt_ss_y=[0, self.depletion(R_ss[0], self.S, self.kvalues)])#y parallel
        self.data_dR_dt_R_point_lab.data = dict(R_ss=[R_ss], dR_dt_ss=[self.depletion(R_ss, self.S, self.kvalues)], lab=["R_ss = %.2f" % R_ss]) #label
        self.data_dR_dt_R_point.data = dict(R_ss=R_ss, dR_dt_ss=self.depletion(R_ss, self.S, self.kvalues))
          
        S = np.linspace(self.Smin, self.Smax, self.N)
        R_y = self.Rss(S, self.kvalues)
        self.data_S_R.data = dict(S=S, R=R_y)
        self.data_S_R_lines.data=dict(R_ss_y=[0, R_ss[0]], S_ss_y=[self.S, self.S], #y parallel
                                      R_ss_x=[R_ss[0], R_ss[0]], S_ss_x=[0, self.S]) #x parallel
        self.data_S_R_point.data = dict(S_ss=S0, R_ss=R_ss)#dot
    
    def plot(self, doc):
        for w in self.k_var:
            w.on_change('value', self.update_data)
        self.S_var.on_change('value', self.update_data)
        
        # Set up layouts and add to document
        plot_dR_dt, plot_R_S, plot_wire, plot_formula = self.create_figure()
        l = row([column([plot_wire, plot_formula, widgetbox(self.S_var), row([widgetbox(self.k_var[0::2], width=150), widgetbox(self.k_var[1::2], width=150)])]),
                 plot_dR_dt, plot_R_S], sizing_mode='fixed', width=1500, height=800)
        doc.add_root(l)
    
    def show_notebook(self):
        handler = FunctionHandler(self.plot)
        app = Application(handler)
        show(app, notebook_url="localhost:8888")
        
    def show_server(self):
        self.plot(curdoc())
        curdoc().title = self.title 
        
    
    

In [124]:
def Rss (S, k):
    return (k['value'][k['name']=='k0'] + k['value'][k['name']=='k1']*S)/k['value'][k['name']=='k2'] + S*0

def depletion(R, S, k):
    return k['value'][k['name']=='k2']*R

def aggregation(R, S, k):
    return k['value'][k['name']=='k0'] + k['value'][k['name']=='k1']*S +0*R

k = np.array([('k0', 0.01), ('k1', 1.0), ('k2', 5.0)],
             dtype=[('name', 'U10'), ('value', 'f4')])

a1 = plotter(aggregation, depletion, Rss, _S_0=1,
                 _kvalues=k, _wire_url='img/wire1a.png', _formula_url='img/diff1a.png',
                _N=200, _Rmax=2, _Rmin=0, _Smax=3, _Smin=0, _title='Figure 1a', _imgheight=150)
a1.show_notebook()

In [99]:
def Rss (S, k):
    return (k['value'][k['name']=='R_T']*S)/(k['value'][k['name']=='k2'] / k['value'][k['name']=='k1'] + S)

def depletion(R, S, k):
    return k['value'][k['name']=='k2']*R

def aggregation(R, S, k):
    return k['value'][k['name']=='k1'] * S * (k['value'][k['name']=='R_T'] - R)

k = np.array([('R_T', 1.0), ('k1', 1.0), ('k2', 1.0)],
             dtype=[('name', 'U10'), ('value', 'f4')])

b1 = plotter(aggregation, depletion, Rss, _S_0=1,
                 _kvalues=k, _wire_url='img/wire1b.png', _formula_url='img/diff1b.png',
                _N=200, _Rmax=1, _Rmin=0, _Smax=3, _Smin=0, _title='Figure 1b', _imgheight=150)
b1.show_notebook()

In [107]:
def G(u, v, J, K):
    return 2*u*K/(v - u + v*J + u*K + np.sqrt((v - u + v*J + u*K)**2 - 4*(v - u)*u*K))

def Rss(S, k):
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    km1 = k['value'][k['name']=='km1']
    km2 = k['value'][k['name']=='km2']
    R_T = k['value'][k['name']=='R_T']
    return R_T * G(k1*S, k2, km1/R_T, km2/R_T)

def depletion(R, S, k):
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    km1 = k['value'][k['name']=='km1']
    km2 = k['value'][k['name']=='km2']
    R_T = k['value'][k['name']=='R_T']
    return k2*R / (km2 + R)

def aggregation(R, S, k):
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    km1 = k['value'][k['name']=='km1']
    km2 = k['value'][k['name']=='km2']
    R_T = k['value'][k['name']=='R_T']
    return k1*S*(R_T - R) / (km1 + R_T - R)

k = np.array([('R_T', 1.0), ('k1', 1.0), ('k2', 1.0), ('km1', 0.05), ('km2', 0.05)],
             dtype=[('name', 'U10'), ('value', 'f4')])

c1 = plotter(aggregation, depletion, Rss, _S_0=1,
                 _kvalues=k, _wire_url='img/wire1c.png', _formula_url='img/diff1c.png',
                _N=200, _Rmax=1, _Rmin=0, _Smax=3, _Smin=0, _title='Figure 1c', _imgheight=130)
c1.show_notebook()

In [125]:
def Rss(S, k):
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    k3 = k['value'][k['name']=='k3']
    k4 = k['value'][k['name']=='k4']
    return k1*k4/k2/k3 + 0*S

def Xss(S, k):
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    k3 = k['value'][k['name']=='k3']
    k4 = k['value'][k['name']=='k4']
    return k3*S/k4 

def depletion(R, S, k):
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    k3 = k['value'][k['name']=='k3']
    k4 = k['value'][k['name']=='k4']
    return k2*Xss(S, k)*R

def aggregation(R, S, k):
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    k3 = k['value'][k['name']=='k3']
    k4 = k['value'][k['name']=='k4']
    return k1*S + 0*R

k = np.array([('k1', 2.0), ('k2', 2.0), ('k3', 1.0), ('k4', 1.0)],
             dtype=[('name', 'U10'), ('value', 'f4')])

c1 = plotter(aggregation, depletion, Rss, _S_0=1,
                 _kvalues=k, _wire_url='img/wire1d.png', _formula_url='img/diff1d.png',
                _N=200, _Rmax=2, _Rmin=0, _Smax=3, _Smin=0, _title='Figure 1d', _imgheight=300)
c1.show_notebook()

In [102]:
#implement class plotter that plots everything that is necessary
class plotter_bif:
    def __init__(self, _aggr, _depl, _dRdt, _S_0,
                 _kvalues=np.ones(7), _kmax=0, _wire_url=0, _formula_url=0,
                _N=200, _Rmax=2, _Rmin=0, _Smax=3, _Smin=0, _title='test', _R0=0, _imgheight=150):
        self.aggregation = _aggr
        self.depletion = _depl
        self.dRdt = _dRdt
        self.kvalues = _kvalues
        self.N = _N #density of plot points
        self.Rmax = _Rmax
        self.Rmin = _Rmin
        self.Smax = _Smax
        self.Smin = _Smin
        self.S = _S_0
        S0 = [_S_0] #this is necessary since some struct need a list as input
        self.wire_url = _wire_url
        self.formula_url = _formula_url
        self.title = _title
        self.R = _R0
        self.imgheight=_imgheight
        
        if (np.size(_kmax)==1):
            self.kmax=np.array(_kvalues)
            self.kmax[:]['value'] = self.kmax[:]['value']*2
        else:
            self.kmax=_kmax
            
        
        
        # Set up data
        R_x = np.linspace(self.Rmin, self.Rmax, self.N)
        self.dRmin = np.max(self.depletion(R_x, self.kvalues))
        dR_dt_depl = self.depletion(R_x, self.kvalues)
        dR_dt_aggr = self.aggregation(R_x, self.S, self.kvalues)
        #R_ss = self.Rss(self.S, self.kvalues)
        #dR_dt_ss = self.depletion(R_ss, self.kvalues)
        self.dRmax = max(np.max(dR_dt_depl), np.max(dR_dt_aggr))
        self.dRmin = min(np.min(dR_dt_depl), np.min(dR_dt_aggr))
        
        self.data_dR_dt_R = ColumnDataSource(data=dict(R=R_x, dR_dt_depl=dR_dt_depl, dR_dt_aggr=dR_dt_aggr)) #graphs
        #self.data_dR_dt_R_lines = ColumnDataSource(data=dict(R_ss_y=[R_ss, R_ss], dR_dt_ss_y=[0, depletion(R_ss, self.kvalues)]))#y parallel
        #self.data_dR_dt_R_point_lab = ColumnDataSource(data=dict(R_ss=[R_ss], dR_dt_ss=[dR_dt_ss], lab=["R_ss = %.2f" % R_ss])) #label
        #self.data_dR_dt_R_point = ColumnDataSource(data=dict(R_ss=R_ss, dR_dt_ss=dR_dt_ss)) #label
        #we need to separate this, since bokeh has problem with drawing circles from data in the label structure
        
        #2nd graph, this differs from original graph
        R_y = np.linspace(self.Rmin, self.Rmax, self.N)
        S_x = np.linspace(self.Smin, self.Smax, self.N)
        
        R_, S_ = np.meshgrid(R_y, S_x)
        RS = self.dRdt(R_, S_, self.kvalues)
        R_split = self.split_R(RS, R_)
        S_split = np.split(S_x, self.get_index_Scrit(RS))
        #self.data_S_R = ColumnDataSource(data=dict(R_split=R_split, S_split=S_split))
        
        #implement a list of columnnames and use that list with add to append datasource
        self.data_R_S = []
        self.tags_R_S = []
        for i, r in enumerate(R_split):
            data_ = ColumnDataSource()
            s = S_split[i]
            tag = 's%i' % i
            data_.add(s, tag)
            self.tags_R_S.append([tag])
            for j, _r in  enumerate(r):
                tag = 'r%i' % j
                data_.add(_r, tag)
                self.tags_R_S[i].append(tag)
            self.data_R_S.append(data_)
        
        
        # Set up widgets
        self.S_var = Slider(title="S", value=self.S, start=self.Smin, end=self.Smax, step=(self.Smax - self.Smin)/100)
        self.k_var = list(np.empty(np.size(self.kvalues)))
        for i, k in enumerate(self.kvalues):
            self.k_var[i] = Slider(title=k['name'], value=k['value'], start=0.0, end=self.kmax[i]['value'], step=0.01)            
            
    def signchange2D(self, a):
        asign = np.sign(a)
        return ((np.roll(asign, 1, axis=1) - asign) != 0).astype(int)
    
    def signchange(self, a):
        asign = np.sign(a)
        return ((np.roll(asign, 1) - asign) != 0).astype(int)
    
    def corr_sign(self, a):
        signed = self.signchange2D(a)
        #print(signed[300])
        #get rid of preceeeding 1 due to wrap around -> correct all 1s at [0] and then roll left
        signed[:, 0] = 0
        signed = np.roll(signed, -1, axis=1)
        return signed
    
    #get indices that have equal numbers of root / get subarrays
    def get_index_Scrit(self, a):
        roots = np.sum(self.corr_sign(a), axis=1)
        #print(roots)
        root_indeces = np.array([0])
        #the first index is always the beginning of a subarray
        
        #use np.roll to find 'signchanges' in mask
        for n in np.unique(roots):
            mask = np.isin(roots, n)
            mask = mask.astype(int)
            changes = self.signchange(mask)
            #changes[:, 0] = 1
            #mark first element, as the first element of each subarray is marked
            indeces = np.argwhere(changes==1)
            root_indeces = np.append(root_indeces, indeces)
            
        return np.unique(root_indeces)[1:]
        #return only unique indeces, since some indeces might appear more often
        #also 0 index is not needed
        
    def split_R(self, RS, R_):
        #if np.size(self.get_index_Scrit(RS))==0:
            #return R_
        Rs = R_[self.corr_sign(RS)==1]
        N = np.size(RS, 0)
        result = []
        old_n = 0
        first_ind = 0
        for n in np.append(self.get_index_Scrit(RS), N):
            R = np.empty(n-old_n)
            m = np.sum(self.corr_sign(RS), axis=1)[n-1]
            #get #roots
            #print(first_ind, old_n, m, n)
            for i in range(m):
                #print(i, m, n, first_ind, old_n , np.shape(R), np.shape(Rs), np.shape(Rs[first_ind+i:first_ind+(n-old_n)*m+i:m]))
                #R = np.vstack((R, Rs[first_ind+i:first_ind+m*n+i:m]))
                R = np.vstack((R, Rs[first_ind+i:first_ind+(n-old_n)*m+i:m]))
                #start:stop:step, stop is not inclusive
            result.append(R[1:])
            #get rid of preceeding zeros
            #first_ind = first_ind+m*n
            first_ind = first_ind+(n-old_n)*m
            old_n = n
        return result
    
            
    def create_figure(self):
        # Set up plot
        #plot_dR_dt = figure(plot_height=600, plot_width=600, title="Depletion/Aggregation Rate",
        #      #tools="crosshair,pan,reset,save,wheel_zoom",
        #      x_range=[self.Rmin, self.Rmax], y_range=[self.dRmin, self.dRmax], toolbar_location="above")
        

        plot_dR_dt = figure(plot_height=800, plot_width=600, title="Depletion/Aggregation Rate",
              x_range=[self.Rmin, self.Rmax], y_range=[self.dRmin, self.dRmax])
        
        
        plot_dR_dt.line('R', 'dR_dt_aggr', source=self.data_dR_dt_R, line_width=3, line_alpha=0.6, color='steelblue')
        plot_dR_dt.line('R', 'dR_dt_depl', source=self.data_dR_dt_R, line_width=3, line_alpha=0.6, color='crimson')
        #plot_dR_dt.line('R_ss_y', 'dR_dt_ss_y', source=self.data_dR_dt_R_lines, line_width=3, line_alpha=0.6, color='red', line_dash='4 4')
        
        #labels = LabelSet(x='R_ss', y='dR_dt_ss', text='lab', level='glyph',
        #      x_offset=7, y_offset=-25, source=self.data_dR_dt_R_point_lab, render_mode='canvas')
        #plot_dR_dt.add_layout(labels)
        
        #plot_dR_dt.circle('R_ss', 'dR_dt_ss', source=self.data_dR_dt_R_point, fill_color="white", size=10)
        
        plot_dR_dt.yaxis.axis_label = "dR/dt"
        plot_dR_dt.xaxis.axis_label = "R"
        
        plot_dR_dt.toolbar.logo = None
        plot_dR_dt.toolbar_location = None
        
            
        #this part differs to original class 
        plot_R_S= figure(plot_height=800, plot_width=600, title="Steady State Solutions",
                  x_range=[self.Smin, self.Smax], y_range=[self.Rmin, self.Rmax], toolbar_location="above")
        
        for i, x in enumerate(self.tags_R_S):
            s = x[0]
            for j, y in enumerate(x[1:]):
                if j%2 == 1:
                    plot_R_S.line(x=s, y=y, source=self.data_R_S[i], line_width=3, line_alpha=0.6, line_dash='dotted', color='black')        
                else:
                    plot_R_S.line(x=s, y=y, source=self.data_R_S[i], line_width=3, line_alpha=0.6, color='black')        
        #plot_R_S.circle(x='S_ss', y='R_ss', source=self.data_S_R_point, fill_color="white", size=10)
        
        plot_R_S.yaxis.axis_label = "R_ss"
        plot_R_S.xaxis.axis_label = "S"
        
        plot_R_S.toolbar.logo = None
        plot_R_S.toolbar_location = None
        
        plot_wire = figure(plot_height=300, plot_width=300, x_range=(0,1), y_range=(0,1), title='Wire Diagramm')
        plot_wire.image_url(url=[self.wire_url], x=0, y=1, w=1, h=1)
        
        plot_wire.toolbar.logo = None
        plot_wire.toolbar_location = None
        plot_wire.axis.visible = False
        
        plot_formula = figure(plot_width=300, plot_height=self.imgheight, x_range=(0,1), y_range=(0,1), title='Formulas')
        plot_formula.image_url(url=[self.formula_url], x=.01, y=.99, w=.99, h=.99)
        
        plot_formula.toolbar.logo = None
        plot_formula.toolbar_location = None
        plot_formula.axis.visible = False
        
        return plot_dR_dt, plot_R_S, plot_wire, plot_formula 
        
    def update_data(self, attrname, old, new):
    
        # Get the current slider values
        self.S = self.S_var.value
        S0 = [self.S]
        for i,k in enumerate(self.k_var):
             self.kvalues['value'][i] = k.value
    
        ## update curve
        R_x = np.linspace(self.Rmin, self.Rmax, self.N)
        dR_dt_depl = self.depletion(R_x, self.kvalues)
        dR_dt_aggr = self.aggregation(R_x, self.S, self.kvalues)
        #R_ss = Rss(self.S, self.kvalues)
        
        self.data_dR_dt_R.data = dict(R=R_x, dR_dt_depl=dR_dt_depl, dR_dt_aggr=dR_dt_aggr) #graphs
        #self.data_dR_dt_R_lines.data = dict(R_ss_y=[R_ss, R_ss], dR_dt_ss_y=[0, depletion(R_ss, self.kvalues)])#y parallel
        #self.data_dR_dt_R_point_lab.data = dict(R_ss=[R_ss], dR_dt_ss=[depletion(R_ss, self.kvalues)], lab=["R_ss = %.2f" % R_ss]) #label
        #self.data_dR_dt_R_point.data = dict(R_ss=R_ss, dR_dt_ss=depletion(R_ss, self.kvalues))
          
        #S = np.linspace(self.Smin, self.Smax, self.N)
        #R_y = Rss(S, self.kvalues)
        
        R_y = np.linspace(self.Rmin, self.Rmax, self.N)
        S_x = np.linspace(self.Smin, self.Smax, self.N)
        
        R_, S_ = np.meshgrid(R_y, S_x)
        RS = self.dRdt(R_, S_, self.kvalues)
        R_split = self.split_R(RS, R_)
        S_split = np.split(S_x, self.get_index_Scrit(RS))
        
        #implement a list of column_names and use that list with add to append datasource
        for i, r in enumerate(R_split):
            s = S_split[i]
            tag = 's%i' % i
            self.data_R_S[i].data[tag] = s
            self.tags_R_S[i][0] = tag
            for j, _r in  enumerate(r):
                tag = 'r%i' % j
                self.data_R_S[i].data[tag] = _r
                self.tags_R_S[i][j+1] = tag
        
        #implement a way to add or remove entries to the datastructure for changes in bifurcation number
        
        #self.data_R_S = []
        #self.tags_R_S = []
        #for i, r in enumerate(R_split):
        #    data_ = ColumnDataSource()
        #    s = S_split[i]
        #    tag = 's%i' % i
        #    data_.add(s, tag)
        #    self.tags_R_S.append([tag])
        #    for j, _r in  enumerate(r):
        #        tag = 'r%i' % j
        #        data_.add(_r, tag)
        #        self.tags_R_S[i].append(tag)
        #    self.data_R_S.append(data_)
        
        #self.data_S_R_lines.data=dict(R_ss_y=[0, R_ss], S_ss_y=[self.S, self.S], #y parallel
        #                              R_ss_x=[R_ss, R_ss], S_ss_x=[0, self.S]) #x parallel
        #self.data_S_R_point.data = dict(S_ss=S0, R_ss=R_ss)#dot
    
    def plot(self, doc):
        for w in self.k_var:
            w.on_change('value', self.update_data)
        self.S_var.on_change('value', self.update_data)
        
        # Set up layouts and add to document
        plot_dR_dt, plot_R_S, plot_wire, plot_formula = self.create_figure()
        l = row([column([plot_wire, plot_formula, widgetbox(self.S_var), row([widgetbox(self.k_var[0::2], width=150), widgetbox(self.k_var[1::2], width=150)])]),
                 plot_dR_dt, plot_R_S], sizing_mode='fixed', width=1500, height=800)
        doc.add_root(l)
    
    def show_notebook(self):
        handler = FunctionHandler(self.plot)
        app = Application(handler)
        show(app, notebook_url="localhost:8888")
        
    def show_server(self):
        self.plot(curdoc())
        curdoc().title = self.title 

In [56]:
#implement class plotter that plots everything that is necessary
class plotter_bif2:
    def __init__(self, _aggr, _depl, _dRdt, _S_0,
                 _kvalues=np.ones(7), _wire_url=0, _formula_url=0,
                _N=200, _Rmax=2, _Rmin=0, _Smax=3, _Smin=0, _title='test', _R0=0, _kmax=np.zeros(7), _imgheight=150):
        self.aggregation = _aggr
        self.depletion = _depl
        self.dRdt = _dRdt
        self.kvalues = _kvalues
        self.N = _N #density of plot points
        self.Rmax = _Rmax
        self.Rmin = _Rmin
        self.Smax = _Smax
        self.Smin = _Smin
        self.S = _S_0
        S0 = [_S_0] #this is necessary since some struct need a list as input
        self.wire_url = _wire_url
        self.formula_url = _formula_url
        self.title = _title
        self.R = _R0
        self.imgheight= _imgheight
        if (_kmax==np.zeros(7)).all:
            self.kmax=np.array(_kvalues)
            self.kmax[:]['value'] = self.kmax[:]['value']*2
        else:
            self.kmax=_kmax
        
        
        # Set up data
        R_x = np.linspace(self.Rmin, self.Rmax, self.N)
        self.dRmin = np.max(self.depletion(R_x, self.kvalues))
        dR_dt_depl = self.depletion(R_x, self.kvalues)
        dR_dt_aggr = self.aggregation(R_x, self.S, self.kvalues)
        #R_ss = self.Rss(self.S, self.kvalues)
        #dR_dt_ss = self.depletion(R_ss, self.kvalues)
        self.dRmax = max(np.max(dR_dt_depl), np.max(dR_dt_aggr))
        self.dRmin = min(np.min(dR_dt_depl), np.min(dR_dt_aggr))
        
        self.data_dR_dt_R = ColumnDataSource(data=dict(R=R_x, dR_dt_depl=dR_dt_depl, dR_dt_aggr=dR_dt_aggr)) #graphs
        #self.data_dR_dt_R_lines = ColumnDataSource(data=dict(R_ss_y=[R_ss, R_ss], dR_dt_ss_y=[0, depletion(R_ss, self.kvalues)]))#y parallel
        #self.data_dR_dt_R_point_lab = ColumnDataSource(data=dict(R_ss=[R_ss], dR_dt_ss=[dR_dt_ss], lab=["R_ss = %.2f" % R_ss])) #label
        #self.data_dR_dt_R_point = ColumnDataSource(data=dict(R_ss=R_ss, dR_dt_ss=dR_dt_ss)) #label
        #we need to separate this, since bokeh has problem with drawing circles from data in the label structure
        
        #2nd graph, this differs from original graph
        R_y = np.linspace(self.Rmin, self.Rmax, self.N)
        S_x = np.linspace(self.Smin, self.Smax, self.N)
        
        R_, S_ = np.meshgrid(R_y, S_x)
        RS = self.dRdt(R_, S_, self.kvalues)
        R_split = self.split_R(RS, R_)
        S_split = np.split(S_x, self.get_index_Scrit(RS))
        #self.data_S_R = ColumnDataSource(data=dict(R_split=R_split, S_split=S_split))
        
        #implement a list of columnnames and use that list with add to append datasource
        self.data_R_S = []
        self.tags_R_S = []
        for i, r in enumerate(R_split):
            data_ = ColumnDataSource()
            s = S_split[i]
            tag = 's%i' % i
            data_.add(s, tag)
            self.tags_R_S.append([tag])
            for j, _r in  enumerate(r):
                #print(r, _r)
                tag = 'r%i' % j
                data_.add(_r, tag)
                self.tags_R_S[i].append(tag)
            self.data_R_S.append(data_)
        
        
        # Set up widgets
        self.S_var = Slider(title="S", value=self.S, start=self.Smin, end=self.Smax, step=(self.Smax - self.Smin)/100)
        self.k_var = list(np.empty(np.size(self.kvalues)))
        for i, k in enumerate(self.kvalues):
            #self.k_var[i] = Slider(title=k['name'], value=k['value'], start=0.0, end=2.0, step=0.01)
            self.k_var[i] = Slider(title=k['name'], value=k['value'], start=0.0, end=self.kmax[i]['value'], step=0.01)            
            
            
    def signchange2D(self, a):
        #print(a[300])
        asign = np.sign(a)
        return ((np.roll(asign, 1, axis=1) - asign) != 0).astype(int)
    
    def signchange(self, a):
        asign = np.sign(a)
        return ((np.roll(asign, 1) - asign) != 0).astype(int)
    
    def corr_sign(self, a):
        signed = self.signchange2D(a)
        #print(signed[300])
        #get rid of preceeeding 1 due to wrap around -> correct all 1s at [0] and then roll left
        signed[:, 0] = 0
        signed = np.roll(signed, -1, axis=1)
        return signed
    
    #get indices that have equal numbers of root / get subarrays
    def get_index_Scrit(self, a):
        roots = np.sum(self.corr_sign(a), axis=1)
        #print(roots)
        root_indeces = np.array([0])
        #the first index is always the beginning of a subarray
        
        #use np.roll to find 'signchanges' in mask
        for n in np.unique(roots):
            mask = np.isin(roots, n)
            mask = mask.astype(int)
            changes = self.signchange(mask)
            #changes[:, 0] = 1
            #mark first element, as the first element of each subarray is marked
            indeces = np.argwhere(changes==1)
            root_indeces = np.append(root_indeces, indeces)
            
        return np.unique(root_indeces)[1:]
        #return only unique indeces, since some indeces might appear more often
        #also 0 index is not needed
        
    def split_R(self, RS, R_):
        #if np.size(self.get_index_Scrit(RS))==0:
            #return R_
        Rs = R_[self.corr_sign(RS)==1]
        N = np.size(RS, 0)
        result = []
        old_n = 0
        first_ind = 0
        for n in np.append(self.get_index_Scrit(RS), N):
            R = np.empty(n-old_n)
            m = np.sum(self.corr_sign(RS), axis=1)[n-1]
            #get #roots
            #print(first_ind, old_n, m, n)
            for i in range(m):
                #print(i, m, n, first_ind, old_n , np.shape(R), np.shape(Rs), np.shape(Rs[first_ind+i:first_ind+(n-old_n)*m+i:m]))
                #R = np.vstack((R, Rs[first_ind+i:first_ind+m*n+i:m]))
                R = np.vstack((R, Rs[first_ind+i:first_ind+(n-old_n)*m+i:m]))
                #start:stop:step, stop is not inclusive
            result.append(R[1:])
            #get rid of preceeding zeros
            #first_ind = first_ind+m*n
            first_ind = first_ind+(n-old_n)*m
            old_n = n
        return result
    
            
    def create_figure(self):
        # Set up plot
        #plot_dR_dt = figure(plot_height=600, plot_width=600, title="Depletion/Aggregation Rate",
        #      #tools="crosshair,pan,reset,save,wheel_zoom",
        #      x_range=[self.Rmin, self.Rmax], y_range=[self.dRmin, self.dRmax], toolbar_location="above")

        plot_dR_dt = figure(plot_height=600, plot_width=600, title="Depletion/Aggregation Rate",
              x_range=[self.Rmin, self.Rmax], y_range=[self.dRmin, self.dRmax])
        
        
        plot_dR_dt.line('R', 'dR_dt_aggr', source=self.data_dR_dt_R, line_width=3, line_alpha=0.6, color='steelblue')
        plot_dR_dt.line('R', 'dR_dt_depl', source=self.data_dR_dt_R, line_width=3, line_alpha=0.6, color='crimson')
        #plot_dR_dt.line('R_ss_y', 'dR_dt_ss_y', source=self.data_dR_dt_R_lines, line_width=3, line_alpha=0.6, color='red', line_dash='4 4')
        
        #labels = LabelSet(x='R_ss', y='dR_dt_ss', text='lab', level='glyph',
        #      x_offset=7, y_offset=-25, source=self.data_dR_dt_R_point_lab, render_mode='canvas')
        #plot_dR_dt.add_layout(labels)
        
        #plot_dR_dt.circle('R_ss', 'dR_dt_ss', source=self.data_dR_dt_R_point, fill_color="white", size=10)
        
        plot_dR_dt.yaxis.axis_label = "dR/dt"
        plot_dR_dt.xaxis.axis_label = "R"
        
        plot_dR_dt.toolbar.logo = None
        plot_dR_dt.toolbar_location = None
        
            
        #this part differs to original class 
        plot_R_S= figure(plot_height=600, plot_width=600, title="Steady State Solutions",
                  x_range=[self.Smin, self.Smax], y_range=[self.Rmin, self.Rmax], toolbar_location="above")
        
        for i, x in enumerate(self.tags_R_S):
            s = x[0]
            for j, y in enumerate(x[1:]):
                if j%2 == 1:
                    plot_R_S.line(x=s, y=y, source=self.data_R_S[i], line_width=3, line_alpha=0.6, line_dash='dotted', color='black')        
                else:
                    plot_R_S.line(x=s, y=y, source=self.data_R_S[i], line_width=3, line_alpha=0.6, color='black')        
        #plot_R_S.circle(x='S_ss', y='R_ss', source=self.data_S_R_point, fill_color="white", size=10)
        
        plot_R_S.yaxis.axis_label = "R_ss"
        plot_R_S.xaxis.axis_label = "S"
        
        plot_R_S.toolbar.logo = None
        plot_R_S.toolbar_location = None
        
        plot_wire = figure(plot_height=200, plot_width=200, x_range=(0,1), y_range=(0,1), title='Wire Diagramm')
        plot_wire.image_url(url=[self.wire_url], x=0, y=1, w=1, h=1)
        
        plot_wire.toolbar.logo = None
        plot_wire.toolbar_location = None
        plot_wire.axis.visible = False
        
        plot_formula = figure(plot_width=200, plot_height=120, x_range=(0,1), y_range=(0,1), title='Formulas')
        plot_formula.image_url(url=[self.formula_url], x=0, y=1, w=1, h=1)
        
        plot_formula.toolbar.logo = None
        plot_formula.toolbar_location = None
        plot_formula.axis.visible = False
        
        return plot_dR_dt, plot_R_S, plot_wire, plot_formula 
        
    def update_data(self, attrname, old, new):
    
        # Get the current slider values
        self.S = self.S_var.value
        S0 = [self.S]
        for i,k in enumerate(self.k_var):
             self.kvalues['value'][i] = k.value
    
        ## update curve
        R_x = np.linspace(self.Rmin, self.Rmax, self.N)
        dR_dt_depl = self.depletion(R_x, self.kvalues)
        dR_dt_aggr = self.aggregation(R_x, self.S, self.kvalues)
        #R_ss = Rss(self.S, self.kvalues)
        
        self.data_dR_dt_R.data = dict(R=R_x, dR_dt_depl=dR_dt_depl, dR_dt_aggr=dR_dt_aggr) #graphs
        #self.data_dR_dt_R_lines.data = dict(R_ss_y=[R_ss, R_ss], dR_dt_ss_y=[0, depletion(R_ss, self.kvalues)])#y parallel
        #self.data_dR_dt_R_point_lab.data = dict(R_ss=[R_ss], dR_dt_ss=[depletion(R_ss, self.kvalues)], lab=["R_ss = %.2f" % R_ss]) #label
        #self.data_dR_dt_R_point.data = dict(R_ss=R_ss, dR_dt_ss=depletion(R_ss, self.kvalues))
          
        #S = np.linspace(self.Smin, self.Smax, self.N)
        #R_y = Rss(S, self.kvalues)
        
        R_y = np.linspace(self.Rmin, self.Rmax, self.N)
        S_x = np.linspace(self.Smin, self.Smax, self.N)
        
        R_, S_ = np.meshgrid(R_y, S_x)
        RS = self.dRdt(R_, S_, self.kvalues)
        R_split = self.split_R(RS, R_)
        S_split = np.split(S_x, self.get_index_Scrit(RS))
        
        #implement a list of column_names and use that list with add to append datasource
        for i, r in enumerate(R_split):
            s = S_split[i]
            tag = 's%i' % i
            self.data_R_S[i].data[tag] = s
            self.tags_R_S[i][0] = tag
            for j, _r in  enumerate(r):
                tag = 'r%i' % j
                self.data_R_S[i].data[tag] = _r
                self.tags_R_S[i][j+1] = tag
        
        #implement a way to add or remove entries to the datastructure for changes in bifurcation number
        
        #self.data_R_S = []
        #self.tags_R_S = []
        #for i, r in enumerate(R_split):
        #    data_ = ColumnDataSource()
        #    s = S_split[i]
        #    tag = 's%i' % i
        #    data_.add(s, tag)
        #    self.tags_R_S.append([tag])
        #    for j, _r in  enumerate(r):
        #        tag = 'r%i' % j
        #        data_.add(_r, tag)
        #        self.tags_R_S[i].append(tag)
        #    self.data_R_S.append(data_)
        
        #self.data_S_R_lines.data=dict(R_ss_y=[0, R_ss], S_ss_y=[self.S, self.S], #y parallel
        #                              R_ss_x=[R_ss, R_ss], S_ss_x=[0, self.S]) #x parallel
        #self.data_S_R_point.data = dict(S_ss=S0, R_ss=R_ss)#dot
    
    def plot(self, doc):
        for w in self.k_var:
            w.on_change('value', self.update_data)
        self.S_var.on_change('value', self.update_data)
        
        # Set up layouts and add to document
        plot_dR_dt, plot_R_S, plot_wire, plot_formula = self.create_figure()
        l = row([column([plot_wire, plot_formula, widgetbox(self.S_var), widgetbox(self.k_var)]),
                 plot_dR_dt, plot_R_S], sizing_mode='fixed', width=1500, height=600)
        doc.add_root(l)
    
    def show_notebook(self):
        handler = FunctionHandler(self.plot)
        app = Application(handler)
        show(app, notebook_url="localhost:8888")
        
    def show_server(self):
        self.plot(curdoc())
        curdoc().title = self.title 


In [66]:
def G(u, v, J, K):
    return 2*u*K/(v - u + v*J + u*K + np.sqrt((v - u + v*J + u*K)**2 - 4*(v - u)*u*K))

def depletion(R, k):
    k0 = k['value'][k['name']=='k0']
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    k3 = k['value'][k['name']=='k3']
    k4 = k['value'][k['name']=='k4']
    J3 = k['value'][k['name']=='J3']
    J4 = k['value'][k['name']=='J4']
    return k2*R

def aggregation(R, S, k):
    k0 = k['value'][k['name']=='k0']
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    k3 = k['value'][k['name']=='k3']
    k4 = k['value'][k['name']=='k4']
    J3 = k['value'][k['name']=='J3']
    J4 = k['value'][k['name']=='J4']
    return k1*S + k0*G(k3*R, k4, J3, J4)

def dRdt(R, S, k):
    return aggregation(R, S, k) - depletion(R, k)

k = np.array([('k0', 0.5), ('k1', 0.01), ('k2', 1.0), ('k3', 1.00), ('k4', 0.2), ('J3', 0.05), ('J4', 0.05)],
             dtype=[('name', 'U10'), ('value', 'f4')])

e1 = plotter_bif(aggregation, depletion, dRdt, _S_0=1,
                 _kvalues=k, _wire_url='img/wire1e.png', _formula_url='img/diff1e.png',
                _N=600, _Rmax=1, _Rmin=0, _Smax=15, _Smin=0, _title='Figure 1e', _imgheight=120)
e1.show_notebook()

In [72]:
def G(u, v, J, K):
    return 2*u*K/(v - u + v*J + u*K + np.sqrt((v - u + v*J + u*K)**2 - 4*(v - u)*u*K))

def depletion(R, k):
    k0 = k['value'][k['name']=='k0']
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    k2_ = k['value'][k['name']=='k2_']
    k3 = k['value'][k['name']=='k3']
    k4 = k['value'][k['name']=='k4']
    J3 = k['value'][k['name']=='J3']
    J4 = k['value'][k['name']=='J4']
    return k2*R + k2_*R*G(k3, k4*R, J3, J4) + 0*R

def aggregation(R, S, k):
    k0 = k['value'][k['name']=='k0']
    k1 = k['value'][k['name']=='k1']
    k2 = k['value'][k['name']=='k2']
    k2_ = k['value'][k['name']=='k2_']
    k3 = k['value'][k['name']=='k3']
    k4 = k['value'][k['name']=='k4']
    J3 = k['value'][k['name']=='J3']
    J4 = k['value'][k['name']=='J4']
    return k1*S + k0 + 0*R

def dRdt(R, S, k):
    return aggregation(R, S, k) - depletion(R, k)

k = np.array([('k0', 0.0), ('k1', 0.05), ('k2', 0.1), ('k2_', 0.5), ('k3', 0.10), ('k4', 0.5), ('J3', 0.05), ('J4', 0.05)],
             dtype=[('name', 'U10'), ('value', 'f4')])
kmax = np.array([('k0', 0.05), ('k1', 1.0), ('k2', 1.), ('k2_', 1.), ('k3', 1.0), ('k4', 1.), ('J3', 0.15), ('J4', 0.15)],
             dtype=[('name', 'U10'), ('value', 'f4')])


f1 = plotter_bif(aggregation, depletion, dRdt, _S_0=1,
                 _kvalues=k, _wire_url='img/wire1f.png', _formula_url='img/diff1f.png',
                _N=600, _Rmax=1, _Rmin=0, _Smax=2, _Smin=0, _title='Figure 1f', _kmax=kmax, _imgheight=100)

f1.show_notebook()

In [88]:
#implement class plotter that plots everything that is necessary
from scipy.integrate import odeint
np.set_printoptions(threshold=10)
from bokeh.palettes import viridis

class plotter_osc:
    def __init__(self, _RX, _RX0, _S_0=0,
                 _kvalues=np.ones(7), _wire_url=0, _formula_url=0,
                 _N=200, _Rmax=2, _Rmin=0, _Smax=3, _Smin=0, _Xmax=1.5, _Xmin=0,
                 _title='test', _R0=0, _quivleng=5, _imgheight=120, _kmax=0):
        self.RX = _RX
        self.RX0 = _RX0
        self.kvalues = _kvalues
        self.N = _N #density of plot points
        self.Rmax = _Rmax
        self.Rmin = _Rmin
        self.Smax = _Smax
        self.Smin = _Smin
        self.Xmax = _Xmax
        self.Xmin = _Xmin
        self.S = _S_0
        S0 = [_S_0] #this is necessary since some struct need a list as input
        self.wire_url = _wire_url
        self.formula_url = _formula_url
        self.title = _title
        self.quivleng = _quivleng
        self.imgheight=_imgheight
        
        if (np.size(_kmax)==1):
            self.kmax=np.array(_kvalues)
            self.kmax[:]['value'] = self.kmax[:]['value']*2
        else:
            self.kmax=_kmax
        
        
        # Set up data
        t = np.arange(0, 80, .1)
        xx = np.linspace(self.Xmin, self.Xmax, self.N)
        rr = np.linspace(self.Rmin, self.Rmax, self.N)
        pars = tuple(np.insert(self.kvalues['value'], 0, self.S))
        y = odeint(_RX, _RX0, t, pars)
        # defines a grid of points
        X, R = np.meshgrid(xx, rr)
        # calculates the value of the derivative at the point in the grid
        dy = self.RX(np.array([X, R]), 0, *pars)
        

        speed = np.sqrt(dy[0]**2 + dy[1]**2)
        theta = np.arctan(dy[1]/dy[0])

        d = 40 #density
        x0 = X[::d, ::d].flatten()
        y0 = R[::d, ::d].flatten()
        length = speed[::d, ::d].flatten()/_quivleng
        angle = theta[::d, ::d].flatten()
        x1 = x0 + length * np.cos(angle)
        y1 = y0 + length * np.sin(angle)
        
        #cm = np.array(["#C7E9B4", "#7FCDBB", "#41B6C4", "#1D91C0", "#225EA8", "#0C2C84"])
        cm = np.array(viridis(6))
        ix = ((length-length.min())/(length.max()-length.min())*5).astype('int')
        colors = cm[ix]
        
        self.data_X_R_traj = ColumnDataSource(data=dict(X=y[:,0], R=y[:,1]))
        self.data_X_R_quiv = ColumnDataSource(data=dict(x0=x0, y0=y0, x1=x1, y1=y1, colors=colors))
        
        #find nullclines like for plotter_bif
        R2, X2 = np.meshgrid(rr, xx)
        dy = self.RX(np.array([X2, R2]), 0, *pars)
        R_split1 = self.split_R(dy[0], R2)
        X_split1 = np.split(xx, self.get_index_Scrit(dy[0]))
        R_split2 = self.split_R(dy[1], R2)
        X_split2 = np.split(xx, self.get_index_Scrit(dy[1]))
        for ind, x in enumerate(np.insert(self.get_index_Scrit(dy[1]), 0, 0)):
            if np.sum(self.corr_sign(dy[1]), axis=1)[x] == 0:
                X_split2 = np.delete(X_split2, ind)
        for ind, x in enumerate(np.insert(self.get_index_Scrit(dy[0]), 0, 0)):
            if np.sum(self.corr_sign(dy[0]), axis=1)[x] == 0:
                X_split1 = np.delete(X_split1, ind)
                
        #self.data_S_R = ColumnDataSource(data=dict(R_split=R_split, S_split=S_split))
        
        #implement a list of columnnames and use that list with add to append datasource
        self.data_X_R_null1 = []
        self.tags_X_R_null1 = []
        for i, r in enumerate(R_split1):
            data_ = ColumnDataSource()
            x = X_split1[i]
            tag = 'x%i' % i
            data_.add(x, tag)
            self.tags_X_R_null1.append([tag])
            for j, _r in  enumerate(r):
                #print(r, _r)
                tag = 'r%i' % j
                data_.add(_r, tag)
                self.tags_X_R_null1[i].append(tag)
            self.data_X_R_null1.append(data_)
        self.data_X_R_null2 = []
        self.tags_X_R_null2 = []
        for i, r in enumerate(R_split2):
            data_ = ColumnDataSource()
            x = X_split2[i]
            tag = 'x%i' % i
            data_.add(x, tag)
            self.tags_X_R_null2.append([tag])
            for j, _r in  enumerate(r):
                #print(r, _r)
                tag = 'r%i' % j
                data_.add(_r, tag)
                self.tags_X_R_null2[i].append(tag)
            self.data_X_R_null2.append(data_)
        
        #use critical indeces to split both graphs even further, then compare all subarrays with one another by subtracting.
        # if the suctraction crosses 0 (signchange) mark that point and add it to coulmndata source
        
        #2nd graph, this differs from original graph
        R_y = np.linspace(self.Rmin, self.Rmax, self.N)
        S_x = np.linspace(self.Smin, self.Smax, self.N)
        
        ymin = []
        ymax = []
        yss = []
        alphapoint = []
        alphaline = []
        S = np.arange(self.Smin, self.Smax, .01)
        t = np.arange(0, 400, .4)
        # loop over the values of s (S)
        for s in S:
            # redefine the parameters using the new S
            pars = tuple(np.insert(self.kvalues['value'], 0, s))
            # integrate again the equation, with new parameters
            y = odeint(_RX, _RX0, t, pars)
            # calculate the minimum and maximum of the populations, but
            # only for the last 200 steps (the long-term solution),
            # appending the result to the list
            ymin.append(y[-500:,:].min(axis=0))
            ymax.append(y[-500:,:].max(axis=0))
            
            #find nullclines like for plotter_bif
            R2, X2 = np.meshgrid(rr, xx)
            dy = self.RX(np.array([X2, R2]), 0, *pars)
            #get length of all points and choose the one with the shortest length
            lengths = dy[0]**2 + dy[1]**2
            yss.append(R2[int(np.argmin(lengths)/np.size(lengths, axis=1)), np.argmin(lengths)%np.size(lengths, axis=1)])
            
            if ymax[-1][1] - ymin[-1][1] < 0.1:
                alphapoint.append(0.0)
                alphaline.append(0.9)
            else:
                alphapoint.append(0.9)
                alphaline.append(0.0)
            #add an alpha value to ymax and ymin depending on how close these values are!
            #do an inverse, that can be used for a line, which should give solid line in the end
            
            
            #R_split1 = self.split_R(dy[0], R2)
            #X_split1 = np.split(xx, self.get_index_Scrit(dy[0]))
            #R_split2 = self.split_R(dy[1], R2)
            #X_split2 = np.split(xx, self.get_index_Scrit(dy[1]))
            #for ind, x in enumerate(np.insert(self.get_index_Scrit(dy[1]), 0, 0)):
            #    if np.sum(self.corr_sign(dy[1]), axis=1)[x] == 0:
            #        X_split2 = np.delete(X_split2, ind)
            #for ind, x in enumerate(np.insert(self.get_index_Scrit(dy[0]), 0, 0)):
            #    if np.sum(self.corr_sign(dy[0]), axis=1)[x] == 0:
            #        X_split1 = np.delete(X_split1, ind)
                
                
        # convert the lists into arrays
        ymin = np.array(ymin)
        ymax = np.array(ymax)
        yss = np.array(yss)
        alphapoint = np.array(alphapoint)
        alphaline = np.array(alphaline)
        
        self.data_S_R = ColumnDataSource(data=dict(S=S, ymin=ymin[:,1], ymax=ymax[:,1], yss=yss[:], alphapoint=alphapoint, alphaline=alphaline))
        
        # Set up widgets
        self.S_var = Slider(title="S", value=self.S, start=self.Smin, end=self.Smax, step=(self.Smax - self.Smin)/100)
        self.k_var = list(np.empty(np.size(self.kvalues)))
        for i, k in enumerate(self.kvalues):
            self.k_var[i] = Slider(title=k['name'], value=k['value'], start=0.0, end=self.kmax[i]['value'], step=0.01)            
    
    def signchange2D(self, a):
        #print(a[300])
        asign = np.sign(a)
        return ((np.roll(asign, 1, axis=1) - asign) != 0).astype(int)
    
    def signchange(self, a):
        asign = np.sign(a)
        return ((np.roll(asign, 1) - asign) != 0).astype(int)
    
    def corr_sign(self, a):
        signed = self.signchange2D(a)
        #get rid of preceeeding 1 due to wrap around -> correct all 1s at [0] and then roll left
        signed[:, 0] = 0
        signed = np.roll(signed, -1, axis=1)
        return signed
    
    #get indices that have equal numbers of root / get subarrays
    def get_index_Scrit(self, a):
        roots = np.sum(self.corr_sign(a), axis=1)
        #print(roots)
        root_indeces = np.array([0])
        #the first index is always the beginning of a subarray
        
        #use np.roll to find 'signchanges' in mask
        for n in np.unique(roots):
            mask = np.isin(roots, n)
            mask = mask.astype(int)
            changes = self.signchange(mask)
            #changes[:, 0] = 1
            #mark first element, as the first element of each subarray is marked
            indeces = np.argwhere(changes==1)
            root_indeces = np.append(root_indeces, indeces)
            
        return np.unique(root_indeces)[1:]
        #return only unique indeces, since some indeces might appear more often
        #also 0 index is not needed
        
    def split_R(self, RS, R_):
        #print(RS)
        #print(R_)
        #if np.size(self.get_index_Scrit(RS))==0: #if there is no critical point, then R is not split
        #    print([R_])
        #    return np.array([R_])
        Rs = R_[self.corr_sign(RS)==1] #Rs contains the y-values at each critical point
        #print(Rs)
        #print('end')
        N = np.size(RS, 0)
        result = []
        old_n = 0
        first_ind = 0
        for n in np.append(self.get_index_Scrit(RS), N):
            R = np.empty(n-old_n)
            m = np.sum(self.corr_sign(RS), axis=1)[n-1]
            if m == 0:
                old_n = n
                continue
            #get #roots
            #print(first_ind, old_n, m, n)
            for i in range(m):
                #print(i, m, n, first_ind, old_n , np.shape(R), np.shape(Rs), np.shape(Rs[first_ind+i:first_ind+(n-old_n)*m+i:m]))
                #R = np.vstack((R, Rs[first_ind+i:first_ind+m*n+i:m]))
                R = np.vstack((R, Rs[first_ind+i:first_ind+(n-old_n)*m+i:m]))
                #start:stop:step, stop is not inclusive
            result.append(R[1:])
            #get rid of preceeding zeros
            #first_ind = first_ind+m*n
            first_ind = first_ind+(n-old_n)*m
            old_n = n
        return result
            
    def create_figure(self):

        plot_X_R = figure(plot_height=800, plot_width=600, title="X-R-Plane",
              x_range=[self.Xmin, self.Xmax], y_range=[self.Rmin, self.Rmax])
        
        plot_X_R.line('X', 'R', source=self.data_X_R_traj, line_width=3, line_alpha=0.6, color='black')
        plot_X_R.segment('x0', 'y0', 'x1', 'y1', color='colors', source=self.data_X_R_quiv, line_width=2)
        
        for i, s in enumerate(self.tags_X_R_null1):
            x = s[0]
            for j, y in enumerate(s[1:]):
                if j%2 == 1:
                    plot_X_R.line(x=x, y=y, source=self.data_X_R_null1[i], line_width=3, line_alpha=0.6, line_dash='dotted', color='crimson')        
                else:
                    plot_X_R.line(x=x, y=y, source=self.data_X_R_null1[i], line_width=3, line_alpha=0.6, color='crimson')  
        
        for i, s in enumerate(self.tags_X_R_null2):
            x = s[0]
            for j, y in enumerate(s[1:]):
                if j%2 == 1:
                    plot_X_R.line(x=x, y=y, source=self.data_X_R_null2[i], line_width=3, line_alpha=0.6, line_dash='dotted', color='steelblue')        
                else:
                    plot_X_R.line(x=x, y=y, source=self.data_X_R_null2[i], line_width=3, line_alpha=0.6, color='steelblue')  
        
        plot_X_R.yaxis.axis_label = "R"
        plot_X_R.xaxis.axis_label = "X"
        
        plot_X_R.toolbar.logo = None
        plot_X_R.toolbar_location = None
        
            
        #this part differs to original class 
        plot_R_S= figure(plot_height=800, plot_width=600, title="Steady State Solutions",
                  x_range=[self.Smin, self.Smax], y_range=[self.Rmin, self.Rmax], toolbar_location="above")
        
        plot_R_S.circle('S', 'ymin', alpha='alphapoint', source=self.data_S_R, color='black')
        plot_R_S.circle('S', 'ymax', alpha='alphapoint', source=self.data_S_R, color='black')
        plot_R_S.line('S', 'ymin', source=self.data_S_R, line_width=3, line_alpha=0.6, color='black')
        plot_R_S.line('S', 'ymax', source=self.data_S_R, line_width=3, line_alpha=0.6, color='black')
        plot_R_S.line('S', 'yss', source=self.data_S_R, line_width=3, line_alpha=0.6, line_dash='dashed', color='black')
        
        plot_R_S.yaxis.axis_label = "R"
        plot_R_S.xaxis.axis_label = "S"
        
        plot_R_S.toolbar.logo = None
        plot_R_S.toolbar_location = None
        
        plot_wire = figure(plot_height=300, plot_width=300, x_range=(0,1), y_range=(0,1), title='Wire Diagramm')
        plot_wire.image_url(url=[self.wire_url], x=0, y=1, w=1, h=1)
        
        plot_wire.toolbar.logo = None
        plot_wire.toolbar_location = None
        plot_wire.axis.visible = False
        
        plot_formula = figure(plot_width=300, plot_height=self.imgheight, x_range=(0,1), y_range=(0,1), title='Formulas')
        plot_formula.image_url(url=[self.formula_url], x=.01, y=.99, w=.98, h=.98)
        
        plot_formula.toolbar.logo = None
        plot_formula.toolbar_location = None
        plot_formula.axis.visible = False
        
        return plot_X_R, plot_R_S, plot_wire, plot_formula 
        
    def update_data(self, attrname, old, new):
    
        # Get the current slider values
        self.S = self.S_var.value
        S0 = [self.S]
        for i,k in enumerate(self.k_var):
             self.kvalues['value'][i] = k.value
    
        ## update curve
        # Set up data
        t = np.arange(0, 80, .1)
        xx = np.linspace(self.Xmin, self.Xmax, self.N)
        rr = np.linspace(self.Rmin, self.Rmax, self.N)
        pars = tuple(np.insert(self.kvalues['value'], 0, self.S))
        y = odeint(self.RX, self.RX0, t, pars)
        # defines a grid of points
        X, R = np.meshgrid(xx, rr)
        # calculates the value of the derivative at the point in the grid
        dy = self.RX(np.array([X, R]), 0, *pars)
        

        speed = np.sqrt(dy[0]**2 + dy[1]**2)
        theta = np.arctan(dy[1]/dy[0])

        d = 40 #density
        x0 = X[::d, ::d].flatten()
        y0 = R[::d, ::d].flatten()
        length = speed[::d, ::d].flatten()/self.quivleng
        angle = theta[::d, ::d].flatten()
        x1 = x0 + length * np.cos(angle)
        y1 = y0 + length * np.sin(angle)
        
        #cm = np.array(["#C7E9B4", "#7FCDBB", "#41B6C4", "#1D91C0", "#225EA8", "#0C2C84"])
        cm = np.array(viridis(6))
        ix = ((length-length.min())/(length.max()-length.min())*5).astype('int')
        colors = cm[ix]
        
        self.data_X_R_traj.data = dict(X=y[:,0], R=y[:,1])
        self.data_X_R_quiv.data = dict(x0=x0, y0=y0, x1=x1, y1=y1, colors=colors)
        
        #find nullclines like for plotter_bif
        R2, X2 = np.meshgrid(rr, xx)
        dy = self.RX(np.array([X2, R2]), 0, *pars)
        R_split1 = self.split_R(dy[0], R2)
        X_split1 = np.split(xx, self.get_index_Scrit(dy[0]))
        R_split2 = self.split_R(dy[1], R2)
        X_split2 = np.split(xx, self.get_index_Scrit(dy[1]))
        for ind, x in enumerate(np.insert(self.get_index_Scrit(dy[1]), 0, 0)):
            if np.sum(self.corr_sign(dy[1]), axis=1)[x] == 0:
                X_split2 = np.delete(X_split2, ind)
        for ind, x in enumerate(np.insert(self.get_index_Scrit(dy[0]), 0, 0)):
            if np.sum(self.corr_sign(dy[0]), axis=1)[x] == 0:
                X_split1 = np.delete(X_split1, ind)
                
        #self.data_S_R = ColumnDataSource(data=dict(R_split=R_split, S_split=S_split))
        
        #implement a list of columnnames and use that list with add to append datasource
        for i, r in enumerate(R_split1):
            x = X_split1[i]
            tag = 'x%i' % i
            self.data_X_R_null1[i].data[tag] = x
            self.tags_X_R_null1[i][0] = tag
            for j, _r in  enumerate(r):
                tag = 'r%i' % j
                self.data_X_R_null1[i].data[tag] = _r
                self.tags_X_R_null1[i][j+1] = tag
        for i, r in enumerate(R_split2):
            x = X_split2[i]
            tag = 'x%i' % i
            self.data_X_R_null2[i].data[tag] = x
            self.tags_X_R_null2[i][0] = tag
            for j, _r in  enumerate(r):
                tag = 'r%i' % j
                self.data_X_R_null2[i].data[tag] = _r
                self.tags_X_R_null2[i][j+1] = tag
                
                
    def plot(self, doc):
        for w in self.k_var:
            w.on_change('value', self.update_data)
        self.S_var.on_change('value', self.update_data)
        
        # Set up layouts and add to document
        plot_dR_dt, plot_R_S, plot_wire, plot_formula = self.create_figure()
        l = row([column([plot_wire, plot_formula, widgetbox(self.S_var), row([widgetbox(self.k_var[0::2], width=150), widgetbox(self.k_var[1::2], width=150)])]),
                 plot_dR_dt, plot_R_S], sizing_mode='fixed', width=1500, height=800)
        doc.add_root(l)
    
    def show_notebook(self):
        handler = FunctionHandler(self.plot)
        app = Application(handler)
        show(app, notebook_url="localhost:8888")
        
    def show_server(self):
        self.plot(curdoc())
        curdoc().title = self.title 

In [89]:
def G(u, v, J, K):
    return 2*u*K/(v - u + v*J + u*K + np.sqrt((v - u + v*J + u*K)**2 - 4*(v - u)*u*K))

def RX(RX, t, S, k0, k1, k2, k2_, k3, k4, k5, k6, J3, J4):
    #k0 = k['value'][k['name']=='k0']
    #k1 = k['value'][k['name']=='k1']
    #k2 = k['value'][k['name']=='k2']
    #k2_ = k['value'][k['name']=='k2_']
    #k3 = k['value'][k['name']=='k3']
    #k4 = k['value'][k['name']=='k4']
    #k5 = k['value'][k['name']=='k5']
    #k6 = k['value'][k['name']=='k6']
    #J3 = k['value'][k['name']=='J3']
    #J4 = k['value'][k['name']=='J4']
    return np.array([   k5*RX[1] - k6*RX[0],
                        k0*G(k3*RX[1], k4, J3, J4) + k1*S - k2*RX[1] - k2_*RX[1]*RX[0]])

RX0 = [1., 1.]

k = np.array([('k0', 4.0), ('k1', 1.0), ('k2', 1.0), ('k2_', 1.0), ('k3', 1.0), ('k4', 1.0), ('k5', 0.1), ('k6', 0.075), ('J3', 0.3), ('J4', 0.3)],
             dtype=[('name', 'U10'), ('value', 'f4')])

f1 = plotter_osc(RX, RX0, _S_0=0.0,
                 _kvalues=k, _wire_url='img/wire2b.png', _formula_url='img/diff2b.png',
                _N=600, _Rmax=2.5, _Rmin=0, _Smax=0.5, _Smin=0, _title='Figure 2b', _quivleng=5)
f1.show_notebook()
#TODO find nullclines in quiver plot

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in true_divide


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:338: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 159), ('r1', 159), ('x0', 143)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 143), ('r1', 159), ('x0', 143)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 177), ('x2', 176)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())),

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 64), ('r1', 64), ('x0', 55)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 55), ('r1', 64), ('x0', 55)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 165), ('x2', 164)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's colum

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 18), ('r1', 18), ('x0', 11)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 11), ('r1', 18), ('x0', 11)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 276), ('r1', 276), ('r2', 276), ('x1', 277)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 19), ('r1', 5), ('x0', 25)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 25), ('r1', 5), ('x0', 25)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 262), ('r1', 262), ('r2', 262), ('x1', 257)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: 

...

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 101), ('r1', 5), ('x0', 106)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 106), ('r1', 5), ('x0', 106)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 196), ('r1', 196), ('r2', 196), ('x1', 193)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarni

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 126), ('r1', 5), ('x0', 129)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 129), ('r1', 5), ('x0', 129)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 177), ('r1', 177), ('r2', 177), ('x1', 175)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarni

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 148), ('r1', 5), ('x0', 151)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 151), ('r1', 5), ('x0', 151)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 161), ('r1', 161), ('r2', 161), ('x1', 159)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarni

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 169), ('r1', 5), ('x0', 172)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 172), ('r1', 5), ('x0', 172)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 146), ('r1', 146), ('r2', 146), ('x1', 143)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarni

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 186), ('r1', 5), ('x0', 189)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 189), ('r1', 5), ('x0', 189)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('r0', 134), ('r1', 134), ('r2', 134), ('x1', 131)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarni

In [16]:
#TODO subsittute lines for points where a bifurcation is present
#TODO find instable steadz states

In [92]:
def G(u, v, J, K):
    return 2*u*K/(v - u + v*J + u*K + np.sqrt((v - u + v*J + u*K)**2 - 4*(v - u)*u*K))

def RX(RX, t, S, k0, k0_, k1, k2, k3, k4, J3, J4):
    #k0 = k['value'][k['name']=='k0']
    #k1 = k['value'][k['name']=='k1']
    #k2 = k['value'][k['name']=='k2']
    #k2_ = k['value'][k['name']=='k2_']
    #k3 = k['value'][k['name']=='k3']
    #k4 = k['value'][k['name']=='k4']
    #k5 = k['value'][k['name']=='k5']
    #k6 = k['value'][k['name']=='k6']
    #J3 = k['value'][k['name']=='J3']
    #J4 = k['value'][k['name']=='J4']
    return np.array([   k1*S - (k0_+k0*G(k3*RX[1], k4, J3, J4))*RX[0],
                        (k0_ + k0*G(k3*RX[1], k4, J3, J4))*RX[0] - k2*RX[1] ])

RX0 = [1., 1.]

k = np.array([('k0', .4), ('k0_', .01), ('k1', 1.0), ('k2', 1.0), ('k3', 1.0), ('k4', .3), ('J3', 0.05), ('J4', 0.05)],
             dtype=[('name', 'U10'), ('value', 'f4')])

g1 = plotter_osc(RX, RX0, _S_0=0.2,
                 _kvalues=k, _wire_url='img/wire2c.png', _formula_url='img/diff2c.png',
                _N=600, _Rmax=1.6, _Rmin=0, _Smax=0.5, _Smin=0, _Xmax=7, _title='Figure 2c', _quivleng=10, _imgheight=130)
g1.show_notebook()
#TODO find nullclines in quiver plot